In [1]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target],
                                      zoom=0.5,
                                      front=[-0.2458, -0.8088, 0.5342],
                                      lookat=[1.7745, 2.2305, 0.9787],
                                      up=[0.3109, -0.5878, -0.7468])



In [10]:
import open3d as o3d
import copy
import numpy as np
print("1. Load two point clouds and show initial pose")
demo_colored_icp_pcds = o3d.data.DemoColoredICPPointClouds()
source = o3d.io.read_point_cloud("0005_norm.ply")
target = o3d.io.read_point_cloud("0006_norm.ply")

# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result_original_color(source, target, current_transformation)

1. Load two point clouds and show initial pose


In [4]:
ls

0001_norm.ply         cap_and_gen_ply.sh*  icp.py             RAFT-Stereo/
0001_transformed.ply  ffph.ipynb           keyboard.py        save_img_tf.py
0003_norm.ply         filter_green.py      monitor_tf.py      tf_ply2.py
0003_transformed.ply  fpfh.py              normal_compute.py  tf_ply.py
0005_norm.ply         gen_pcd.py           old/               viz_ply.py
0006_norm.ply         git.sh*              output/


In [2]:
pip install open3d

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# point to point ICP
current_transformation = np.identity(4)
print("2. Point-to-point ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.2.")
result_icp = o3d.pipelines.registration.registration_icp(
    source, target, 0.2, current_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation)



2. Point-to-point ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.2.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=5.778618e-16, and correspondence_set size of 336004
Access transformation to get result.


In [32]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [500, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation)

3. Colored point cloud registration
[500, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=4.556241e-01, inlier_rmse=2.380102e-02, and correspondence_set size of 960
Access transformation to get result.
[30, 0.02, 1]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=2.988739e-01, inlier_rmse=1.237456e-02, and correspondence_set size of 1911
Access transformation to get result.
[14, 0.01, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=1.712280e-01, inlier_rmse=6.317223e-03, and correspondence_set size of 3366
Access transformation to get result.
